In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import baseline as baseline

In [ ]:
cv_list = [1, 2, 3, 4, 5]

In [ ]:
for cv in cv_list:
    support_df = pd.read_csv('data/few_P300_cv{}_100.csv'.format(cv))
    support_file_list = support_df['data_file'].to_numpy()
    support_label_list = support_df['label'].to_numpy()
    # split dataset
    support_data_df = None
    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(support_file_list, support_label_list):
        support_data_df = support_df.iloc[test_index]
        # break
    print('support data len: {}'.format(len(support_data_df)))

    query_data_df = pd.read_csv('data/test_R300_cv{}_3900.csv'.format(cv))
    print('query data len: {}'.format(len(query_data_df)))

    correct_count = 0
    for q_i, query_data in query_data_df.iterrows():
        q_data = np.loadtxt(query_data['data_file'], delimiter='\t', dtype=float).T
        q_data_y = q_data[1]

        true_label = int(query_data['label'])

        sim_list, support_label_list = [], []
        for s_i, support_data in support_data_df.iterrows():
            s_data = np.loadtxt(support_data['data_file'], delimiter='\t', dtype=float).T
            s_data_y = s_data[1]

            support_label = int(support_data['label'])
            support_label_list.append(support_label)

            # output = baseline.dist_Euclidean(q_data_y, s_data_y)
            output = baseline.dist_Manhattan(q_data_y, s_data_y)
            # output = baseline.dis_cosine(q_data_y, s_data_y)
            sim_list.append(output)

        pred_label_index = sim_list.index(min(sim_list))
        # print(sim_list)

        if true_label == support_label_list[pred_label_index]:
            correct_count += 1
        # print(true_label, support_label_list[pred_label_index])

    print('cv: {}, acc: {:.8f}'.format(cv, correct_count / len(query_data_df)))